[Using Celery With Flask](https://blog.miguelgrinberg.com/post/using-celery-with-flask)  

![](https://blog.miguelgrinberg.com/static/images/flask-celery.png)


- The Celery client. This is used to issue background jobs. When working with Flask, the client runs with the Flask application.
- The Celery workers. These are the processes that run the background jobs. Celery supports local and remote workers, so you can start with a single worker running on the same machine as the Flask server, and later add more workers as the needs of your application grow.
- The message broker. The client communicates with the the workers through a message queue, and Celery supports several ways to implement these queues. The most commonly used brokers are RabbitMQ and Redis.


[github flask-celery-example](https://github.com/miguelgrinberg/flask-celery-example)

![](http://allynh.com/blog/wp-content/uploads/2017/11/Flask_Celery_Redis-676x583.png)

In [3]:
from flask import Flask
from celery import Celery

app = Flask(__name__)
app.config['CELERY_BROKER_URL'] = 'redis://localhost:6379/0'
app.config['CELERY_RESULT_BACKEND'] = 'redis://localhost:6379/0'

celery = Celery(app.name, broker=app.config['CELERY_BROKER_URL'])
celery.conf.update(app.config)

In [6]:
@celery.task
def my_background_task(arg1, arg2):
    # some long running task here
    print(arg1,arg2)
    return result

In [ ]:
task = my_background_task.apply_async(args=[10, 20])

## [Flask:_Multi-Processing_in_Flask](http://wiki.glitchdata.com/index.php/Flask:_Multi-Processing_in_Flask)  


```python
from multiprocessing import Pool
def expensive_function(x):
        # do your expensive time consuming process
        return x*x
if __name__ == '__main__':
        # start 4 worker processes
        pool = Pool(processes=4)
        # evaluate "f(10)" asynchronously
        result = pool.apply_async(expensive_function, [10])
        print result.get(timeout=1)
```        

```python
from multiprocessing import Pool
from flask import Flask

app = Flask(__name__)
_pool = None

def expensive_function(x):
        # import packages that is used in this function
        # do your expensive time consuming process
        return x*x

@app.route('/expensive_calc/<int:x>')
def route_expcalc(x):
        f = _pool.apply_async(expensive_function,[x])
        r = f.get(timeout=2)
        return 'Result is %d'%r

if __name__=='__main__':
        _pool = Pool(processes=4)
        try:
                # insert production server deployment code
                app.run()
        except KeyboardInterrupt:
                _pool.close()
                _pool.join()

```

[Running Multiprocessing in Flask App(Let’s Spawn) Hell Yeah](https://arshpreetsingh.wordpress.com/2017/09/14/running-multiprocessing-in-flask-applets-spawn-hell-yeah/)  

```python
from multiprocessing import Pool
from flask import Flask
from flask import jsonify
import ast
import pandas as pd
import requests
 
app = Flask(__name__)
_pool = None
 
# Function that run multiple tasks
def get_response(x):
"""returns response for URL list"""
m = requests.get((x),verify=False)
return m.text
 
@app.route('/call-me/')
def health_check():
"""returns pandas dataframe into HTML for health-check Services"""
resp_pool = _pool.map(get_response,tasks)
table_frame= pd.DataFrame([ast.literal_eval(resp) for resp in resp_pool])
return table_frame.to_html()
 
if __name__=='__main__':
_pool = Pool(processes=12) # this is important part- We
try:
# insert production server deployment code
app.run(use_reloader=True)
except KeyboardInterrupt:
_pool.close()
_pool.join()
 
```

In [ ]:
from multiprocessing import Pool
from flask import Flask

app = Flask(__name__)
_pool = None

def expensive_function(x):
    # import packages that is used in this function
    # do your expensive time consuming process
    return x*x

def doWork(args):
    si = args[0]
    ei = args[1]
    print(si,ei)
    return si+ei

@app.route('/expensive_calc/<int:x>')
def route_expcalc(x):
#     f = _pool.apply_async(expensive_function,[x])
    f = _pool.map_async(doWork,[(x,x+1),(x+1,x+2),(x+2,x+3),(x+3,x+4)])
    r = f.get(timeout=10)

    print(f)
    print(r)
    return 'Result is '+str(r)
#     r = f.wait(timeout=10)
#     return 'Result is %d'%r

if __name__=='__main__':
    _pool = Pool(processes=4)
    try:
        # insert production server deployment code
        app.run()
    except KeyboardInterrupt:
        _pool.close()
        _pool.join()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


22 23
23 24
21 22
20 21


127.0.0.1 - - [24/Oct/2019 13:57:44] "GET /expensive_calc/20 HTTP/1.1" 200 -


[41, 43, 45, 47]
